### The messy exploration notebook!

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import cv2
import numpy as np
from tqdm import tqdm
from keras.utils import np_utils


# Root directory of the project
ROOT_DIR = os.path.abspath("../")
sys.path.append(ROOT_DIR)  
import helpers

In [ ]:

def crop_and_resize(img, resize_dim=256):
    img=cv2.resize(img,(resize_dim,resize_dim),interpolation=cv2.INTER_AREA)
    return img



def pad(im, desired_size = 256):
    #import pdb; pdb.set_trace()
    old_size = im.shape[:2] # old_size is in (height, width) format
    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])

    # new_size should be in (width, height) format
    im = cv2.resize(im, (new_size[1], new_size[0]))
    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)

    color = [0, 0, 0]
    new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,
        value=color)
    
    return new_im


def get_data(path):
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img=crop_and_resize(img)
    return img

In [ ]:
# Training file directory
DATASET = os.path.join(ROOT_DIR, 'dataset')
OUTPUTS = os.path.join(ROOT_DIR, 'outputs', 'auc')
DATASET, OUTPUTS

In [ ]:
os.listdir(DATASET + "/isic2016")

In [ ]:
# IMAGES PATH
IMAGES = os.path.join(DATASET, 'isic2016', 'ISBI2016_ISIC_Part3_Training_Data')
# GROUND TRUTH PATH
GT = os.path.join(DATASET, 'isic2016', 'ISBI2016_ISIC_Part3_Training_GroundTruth.csv')
len(os.listdir(IMAGES)), len(os.listdir(IMAGES))

In [ ]:
# Read the metadata
df = pd.read_csv(GT, sep=',', names=["FILENAME", "CLASS"])
df.head()

In [ ]:
# filenames and gts
filenames = df['FILENAME'].values
gt = df['CLASS'].values
len(gt)

In [ ]:
filenames[:6]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize= (10, 5))
df['CLASS'].value_counts().plot(kind='bar', ax=ax, title="Histogram of labels", color='g')

helpers.create_directory("{}/outputs".format(ROOT_DIR))
ax.set_xlabel("Class labels")
ax.set_ylabel("Frequency")
#plt.savefig("{}/outputs/numberofclcasses.pdf".format(ROOT_DIR), dpi=300)

In [ ]:
gt[:10]

In [ ]:
filenames[:10]

In [ ]:
labels = []
for s in gt:
    if s == "benign" or s == 0.0 :
        labels.append(0)
    if s == "malignant" or s == 1.0:
        labels.append(1)
        
len(labels)

In [ ]:
labels[:10]

In [ ]:
gt_t = []

for g in labels:
    if g == 0:
        gt_t.append("Benign")
    else:
        gt_t.append("Malignant")
        
gt_t = np.array(gt_t)

gt_t.shape

In [ ]:
import seaborn as sns
sns.set(style="whitegrid", font_scale=1.7)
ax =sns.countplot(x=gt_t, palette="Set2")
ax.set(ylabel='Number of Samples')
plt.grid(False)
plt.savefig("{}/outputs/numberofclcasses.pdf".format(ROOT_DIR), dpi=1000)
plt.show()

In [ ]:
inp_feat = []
g_t = []


cancer = []
non_cancer = []

for f, l in tqdm(zip(filenames[:], labels[:])):
    f = "{}/{}.jpg".format(IMAGES, f)
    img = get_data(f)
    inp_feat.append(img)
    g_t.append(l)
    if l == 1:
        cancer.append(img)
    if l == 0:
        non_cancer.append(img)
    else:
        pass
    img = None

# make nummpy arrays
#inp_feat = np.array(inp_feat)
#g_t = np.array(g_t)

cancer = np.array(cancer)
non_cancer = np.array(non_cancer)

#print(inp_feat.shape, g_t.shape)
print(cancer.shape, non_cancer.shape)

In [ ]:
#inp_feat.shape, g_t.shape

In [ ]:
x = cancer[-20:]

a, b = 4, 5
x = np.reshape(x, (a, b, 256, 256, 3))

y = non_cancer[-20:]
y = np.reshape(y, (a, b, 256, 256, 3))

x.shape, y.shape

In [ ]:
from numpy.random import rand
import matplotlib.pyplot as plt

test_data = x
r, c = test_data.shape[0], test_data.shape[1]
cmaps = [['viridis', 'binary'], ['plasma', 'coolwarm'], ['Greens', 'copper']]

heights = [a[0].shape[0] for a in test_data]
widths = [a.shape[1] for a in test_data[0]]

fig_width = 15.  # inches
fig_height = fig_width * sum(heights) / sum(widths)

f, axarr = plt.subplots(r,c, figsize=(fig_width, fig_height),
        gridspec_kw={'height_ratios':heights})

for i in range(r):
    for j in range(c):
        axarr[i, j].imshow(test_data[i][j])
        axarr[i, j].axis('off')
plt.subplots_adjust(wspace=0, hspace=0, left=0, right=1, bottom=0, top=1)
plt.savefig('{}/{}.pdf'.format("{}/outputs/".format(ROOT_DIR), "cancer"), dpi=1000)
plt.show()

In [ ]:
from numpy.random import rand
import matplotlib.pyplot as plt

test_data = y
r, c = test_data.shape[0], test_data.shape[1]
#test_data = np.array([[cancer[0], cancer[0]],[cancer[0], cancer[0]],[cancer[0], cancer[0]]])
#test_data = np.array([[rand(10,10), rand(10,10)],[rand(5,10), rand(5,10)],[rand(2,10), rand(2,10)]])
cmaps = [['viridis', 'binary'], ['plasma', 'coolwarm'], ['Greens', 'copper']]

heights = [a[0].shape[0] for a in test_data]
widths = [a.shape[1] for a in test_data[0]]

fig_width = 15.  # inches
fig_height = fig_width * sum(heights) / sum(widths)

f, axarr = plt.subplots(r,c, figsize=(fig_width, fig_height),
        gridspec_kw={'height_ratios':heights})

for i in range(r):
    for j in range(c):
        axarr[i, j].imshow(test_data[i][j])
        axarr[i, j].axis('off')
plt.subplots_adjust(wspace=0, hspace=0, left=0, right=1, bottom=0, top=1)
plt.savefig('{}/{}.pdf'.format("{}/outputs/".format(ROOT_DIR), "noncancer"), dpi=1000)
plt.show()

In [ ]:
#helpers.show_images(cancer[:20], cols = 4, titles = None, save_fig = "cancer_samples", path="{}/outputs/".format(ROOT_DIR))

In [ ]:
#helpers.show_images(non_cancer[:20], cols = 4, titles = None, save_fig = "normal_samples", path="{}/outputs/".format(ROOT_DIR))

In [ ]:
# one hot encoded vectors
num_classes = 2
g_t_cat = np_utils.to_categorical(g_t,num_classes)
print(g_t_cat.shape)

In [ ]:
flat = np.array([np.argmax(pred) for pred in g_t_cat])
np.count_nonzero(flat == 1), np.count_nonzero(flat == 0)

In [ ]:
g_t_cat[:4]

In [ ]:
helpers.imshow_group(inp_feat[-50:], g_t_cat[-50:])

In [ ]:
inp_feat.shape, g_t_cat.shape

In [ ]:
img_3d = inp_feat[0]
img_flat = img_3d.flatten()
print(min(img_flat), max(img_flat))

In [ ]:
plt.hist(img_flat, bins=20, color='c')
plt.title("Original input data distribution")
plt.xlabel("Pixel values")
plt.grid(True)
plt.ylabel("Frequency")
plt.show()

In [ ]:
# Test images path
TEST_IMAGES = os.path.join(DATASET, 'isic2016', 'ISBI2016_ISIC_Part3_Test_Data')
# Ground truth path
TEST_GT = os.path.join(DATASET, 'isic2016', 'ISBI2016_ISIC_Part3_Test_GroundTruth.csv')
# Read the metadata
TEST_META = pd.read_csv(TEST_GT, sep=',', names=["FILENAME", "CLASS"])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize= (10, 5))
TEST_META['CLASS'].value_counts().plot(kind='bar', ax=ax, title="Histogram of labels", color='b')

helpers.create_directory("{}/outputs".format(ROOT_DIR))
ax.set_xlabel("Class labels")
ax.set_ylabel("Frequency")
plt.savefig("{}/outputs/test_setnumberofclcasses.png".format(ROOT_DIR), dpi=100)

In [ ]:
# filenames and gts
filenames = TEST_META['FILENAME'].values
gt = TEST_META['CLASS'].values

# convert string labels to numeric values
labels = []
for s in gt:
    if s == "benign" or s == 0.0 :
        labels.append(0)
    if s == "malignant" or s == 1.0:
        labels.append(1)

# all training images and labels     
inp_feat = []
g_t = []

# two classes individually
cancer = []
non_cancer = []

for f, l in tqdm(zip(filenames[:], labels[:])):
    f = "{}/{}.jpg".format(TEST_IMAGES, f)
    img = get_data(f)
    inp_feat.append(img)
    g_t.append(l)

    #----------------
    if l == 1:
        cancer.append(img)
    if l == 0:
        non_cancer.append(img)
    else:
        pass
    #-----------------

    img = None

# make nummpy arrays
inp_feat = np.array(inp_feat)
g_t = np.array(g_t)

# one hot encoded vectors
num_classes = 2
g_t = np_utils.to_categorical(g_t,num_classes)

cancer = np.array(cancer)
non_cancer = np.array(non_cancer)

print(inp_feat.shape, g_t.shape)

In [ ]:
gt = np.array([np.argmax(y) for y in g_t])

gt_t = []

for g in gt:
    if g == 0:
        gt_t.append("Benign")
    else:
        gt_t.append("Malignant")
        
gt_t = np.array(gt_t)

gt_t.shape

In [ ]:
ax =sns.countplot(x=gt, palette="Set2")
ax.set(xlabel='Class Labels', ylabel='Number of Samples')
plt.savefig("{}/outputs/test_numberofclcasses.pdf".format(ROOT_DIR), dpi=300)
plt.show()

In [ ]:
print(cancer.shape, non_cancer.shape)

In [ ]:
helpers.imshow_group(inp_feat[-50:], g_t_cat[-50:])

### Some fancy graphs 

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

labels = ['VGG-GAP \n Baseline', 'VGG-GAP+ \nAugment-5x', 'VGG-GAP+\nAugment-10x', 'VGG-GAP+ \nMeGAN']

auc = [79.08, 78.81, 79.56, 81.18]
postive_misslcass = [55, 51, 47, 22]
sensitivity = [84.46, 85.34, 86.09, 91.76]

x = np.arange(len(labels))  # the label locations
width = 0.28  # the width of the bars


import seaborn as sns
sns.set(style="whitegrid", palette="Set2")
fig, ax = plt.subplots()


rects1 = ax.bar(x - width/2, auc, width, label='AUC')
rects2 = ax.bar(x + width/2, postive_misslcass, width, label='False negatives')
rects3 = ax.bar(x - width/2 * 3.08, sensitivity, width, label='Sensitivity')

ax.set_ylabel('Performance Measure', fontsize=14)
#ax.set_title('Results')
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=0, fontsize=12)
ax.legend(loc='lower left')
ax.set_ylim([0,100])
ax.grid(True)

def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')
        

def autolabel_(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}%'.format(height),
                    xy=( 0.15 + rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')



autolabel_(rects1) # %
autolabel(rects2)
autolabel_(rects3) # %

fig.tight_layout()
plt.grid(False)
plt.savefig("{}/outputs/results.pdf".format(ROOT_DIR), dpi=1200)
plt.show()

In [ ]:
import pickle

def save_obj(obj, name):
    with open('{}/'.format(OUTPUTS) + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


def load_obj(name):
    with open('{}/'.format(OUTPUTS) + name + '.pkl', 'rb') as f:
        return pickle.load(f)

def get_unique(arr):
  '''
  Return unique elements in list
  '''
  arr = set(arr)
  return arr

#raw_features = load_obj('raw_features')
#target = load_obj('target')

In [ ]:
exp1 = load_obj('1')
exp2 = load_obj('2')
exp3 = load_obj('3')
exp4 = load_obj('4')
exp1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp


lw = 1
plt.figure()
labels = ['VGG-GAP Baseline', 'VGG-GAP + Augment-5x', 'VGG-GAP + Augment-10x', 'VGG-GAP + MeGAN']
plt.plot(exp1["fpr"], exp1["tpr"], color='c', lw=lw, linestyle = "--", label='VGG-GAP Baseline (AUC = {0:.2f}%)'.format(exp1["roc_auc"] * 100))
plt.plot(exp2["fpr"], exp2["tpr"], color='b', lw=lw, label='VGG-GAP + Augment-5x (AUC = {0:.2f}%)'.format(exp2["roc_auc"] * 100))
plt.plot(exp3["fpr"], exp3["tpr"], color='g', lw=lw, label='VGG-GAP + Augment-10x (AUC = {0:.2f}%)'.format(exp3["roc_auc"] * 100))
plt.plot(exp4["fpr"], exp4["tpr"], color='r', lw=lw,label='VGG-GAP + MeGAN (AUC = {0:.2f}%)'.format(exp4["roc_auc"] * 100))

plt.plot([0, 1], [0, 1], color='darkorange', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.grid(False)
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.legend(loc="lower right")
plt.savefig("{}/auc.pdf".format(OUTPUTS), dpi=1000)
plt.show()